This notebook demonstrates the result of the first round of data collection, for the Mountain View library <-> Los Altos library round trip.

## Import all the dependencies

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
# For maps
import branca.element as bre

## Load and validate data

The first issue to note is that we actually have two specs here. The first spec is the checked in `evaluation.spec.sample`, which defines calibration for both stationary and moving instances, and some evaluation trips. However, while starting with the calibration, we noticed some inconsistencies between the power curves. So in order to be more consistent, I defined a second, calibration-only spec `examples/calibration.only.json`, which essentially repeats the calibration experiments multiple times.

After that, I returned to the first set of experiments for the moving calibration and the evaluation.

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sdunp = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la")

In [ ]:
pvunp = eipv.PhoneView(sdunp)

In [ ]:
pvunp.validate()

In [ ]:
evunp = eiev.EvaluationView()
evunp.from_view_eval_trips(pvunp, "", "")

In [ ]:
evunp.map("evaluation").keys()

In [ ]:
map_list = ezev.get_map_list_eval_trips(evunp, "evaluation", "")
rows = ezpv.get_row_count(len(map_list), 2)
evaluation_maps = bre.Figure(ratio="{}%".format((rows/4) * 100))
for i, curr_map in enumerate(map_list):
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)

In [ ]:
evaluation_maps

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(10,10))

ezpv.plot_all_power_drain(ios_ax, pvunp.map()["ios"], "evaluation", "")
ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5), ncol=1)
ezpv.plot_all_power_drain(android_ax, pvunp.map()["android"], "evaluation", "")
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5), ncol=1)

### Checking the motion activity

In addition to location data, we also read the motion_activity data from the closed source phone APIs. Let's quickly check how accurate the raw motion activity is.

The medium accuracy runs seem to be much more noisy wrt motion activity transitions. We should really not get a lot of transitions since we essentially took various trains for the entire route. The high accuracy sensing seems to be largely stable, except for one extraneous transition in the middle of the `ucb-sdb-ios-1` run.

In [ ]:
(ifig, ax) = plt.subplots(nrows=2, ncols=1, figsize=(12,8), sharex=True)
ezpv.display_unprocessed_android_activity_transitions(pvunp, ax[0], "evaluation", "")
ezpv.display_unprocessed_ios_activity_transitions(pvunp, ax[1], "evaluation", "")
plt.legend()